## FEATURE ENGINEERING

### Домашняя работа 3

In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv('aac_shelter_outcomes.csv')
data.head(5)

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown


#### 1. Проверка наличия пропусков.

In [3]:
data.isna().sum()

age_upon_outcome        8
animal_id               0
animal_type             0
breed                   0
color                   0
date_of_birth           0
datetime                0
monthyear               0
name                23886
outcome_subtype     42293
outcome_type           12
sex_upon_outcome        2
dtype: int64

In [ ]:
#data[data['animal_id'] == 'A701901']

#### 2. Столбец 'animal_type'

Проанализируем типы животных в датасете.

In [4]:
print(data.animal_type.value_counts())

print('\nДоля собак и кошек:', data.loc[ data['animal_type'].isin(['Dog', 'Cat'])].count().animal_type / 
      data.animal_type.count())

print('Доля остальных животных:', data.loc[ data['animal_type'].isin(['Other', 'Bird', 'Livestock'])].count().animal_type / 
      data.animal_type.count())

Dog          44242
Cat          29422
Other         4249
Bird           334
Livestock        9
Name: animal_type, dtype: int64

Доля собак и кошек: 0.9413207932938049
Доля остальных животных: 0.05867920670619505


Проверим, входят ли собаки и кошки в группу 'Other' столбца 'animal_type'. Если да, то скорректируем тип животного. 

In [5]:
other_animals = data.loc[ data['animal_type'].isin(['Other'])]
other_animals.breed.unique()

array(['Bat Mix', 'Squirrel Mix', 'Bat', 'Raccoon Mix', 'Armadillo Mix',
       'Rat', 'Hamster Mix', 'Raccoon', 'Rabbit Sh Mix', 'Fox Mix',
       'American Sable', 'Opossum Mix', 'Guinea Pig Mix', 'Rex Mix',
       'Rat Mix', 'Rabbit Sh', 'Skunk', 'Tortoise Mix', 'Skunk Mix',
       'Californian Mix', 'Lionhead Mix', 'Opossum', 'Cold Water',
       'Squirrel', 'Guinea Pig', 'Ferret Mix', 'Ferret', 'Rabbit Lh Mix',
       'New Zealand Wht Mix', 'Coyote Mix', 'Lop-Mini Mix', 'Fox',
       'Cottontail Mix', 'Jersey Wooly Mix', 'Turtle Mix', 'American Mix',
       'Lizard Mix', 'Dutch/Rabbit Sh', 'English Spot Mix', 'Armadillo',
       'Lop-Holland Mix', 'Rabbit Sh/Lop-Mini', 'Hamster', 'Lionhead',
       'Dutch Mix', 'Californian', 'Silver Mix', 'Lop-English Mix',
       'Sugar Glider', 'Snake', 'Snake Mix', 'Hotot Mix', 'Rabbit Lh',
       'Mouse Mix', 'Chinchilla-Stnd Mix', 'Checkered Giant Mix',
       'Coyote', 'Netherlnd Dwarf Mix', 'Havana Mix', 'Lop-Mini',
       'Otter Mix', 'Bo

Анализ показал, что в группу  'Other' столбца 'animal_type' собаки и кошки не входят. 

Поскольку тип остальных животных и отношение человека к ним как к домашним сильно отличаются от кошек и собак, это может привести к неточности прогнозирования. Принимая во внимание доля прочих животных (около 6%), примем решение из датасета выделить два: 1-й: кошки и собаки, 2-й: все остальные.

In [10]:
# Создание датасета только из кошек и собак
data = data.loc[ data['animal_type'].isin(['Dog', 'Cat'])]

# Создание датасета только из кошек и собак
data_other = data.loc[ data['animal_type'].isin(['Other', 'Bird', 'Livestock'])]

data.head(5)

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
5,4 months,A664462,Dog,Leonberger Mix,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,2013-10-07T13:06:00,*Edgar,Partner,Transfer,Intact Male


## Анализ датасета только с собаками и кошками

### 1. Столбец 'age_upon_outcome'.

Переведем возраст всех животных в годы. Для этого сначала необходимо извлечь из столбца 'age_upon_outcome' следующие данные: значение возраста и единицы измерения возраста.

In [11]:
# Разделение столбца 'age_upon_outcome' на два столбца 'age' и 'age_period'

data['age'] = data['age_upon_outcome'].str.split(' ').str[0].fillna(0).astype(int)
data['age_period'] = data['age_upon_outcome'].str.split(' ').str[1].fillna(0)

# Удаление столбца 'age_upon_outcome'
data.drop(['age_upon_outcome'], axis=1, inplace=True)

data.head(5)

,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome,age,age_period
0,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male,2,weeks
1,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female,1,year
2,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male,1,year
3,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male,9,years
5,A664462,Dog,Leonberger Mix,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,2013-10-07T13:06:00,*Edgar,Partner,Transfer,Intact Male,4,months


In [8]:
# Уникальные значения столбца 'age_period'
data['age_period'].unique()

array(['weeks', 'year', 'years', 'months', 'month', 'days', 'week', 'day',
       0], dtype=object)

In [13]:
# Перевод возраста всех животных в годы 
def ages(period):
    '''Перевод овзраста животных в годы.'''        
    if period['age_period'] == 'year' or period['age_period'] == 'years':
        return period['age']
    
    if period['age_period'] == 'month' or period['age_period'] == 'months':
        return period['age'] / 12 
    
    if period['age_period'] == 'week' or period['age_period'] == 'weeks':
        return period['age'] / 52
    
    if period['age_period'] == 'day' or period['age_period'] == 'days':
        return period['age'] / 365

data['age_outcome_years_1'] = data.apply(ages, axis=1)

# Удаление столбцов 'age' и 'age_period'
data.drop(['age', 'age_period' ], axis=1, inplace=True)

data.head(5)

,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome,age_outcome_years_1
0,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male,0.038462
1,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female,1.000000
2,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male,1.000000
3,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male,9.000000
5,A664462,Dog,Leonberger Mix,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,2013-10-07T13:06:00,*Edgar,Partner,Transfer,Intact Male,0.333333


Проверим, чтобы возраст всех животных был больше 0.

In [14]:
# Количество и доля животных, возраст которых меньше 0
print('Количество животных с возрастом меньше нуля:', data.query('age_outcome_years_1 < 0')['age_outcome_years_1'].count())
print('Доля животных с возрастом меньше нуля:', data.query('age_outcome_years_1 < 0')['age_outcome_years_1'].count() / data['age_outcome_years_1'].count())


Количество животных с возрастом меньше нуля: 0
Доля животных с возрастом меньше нуля: 0.0


В результате получили столбец с возрастом животных в годах, при этом все значения больше 0.

#### 3. Столбцы 'date_of_birth', 'datetime' и 'monthyear'

In [15]:
# Преобразование метки времени в дату и время
data.loc[:, 'date_of_birth'] = pd.to_datetime(data['date_of_birth'])
data.loc[:, 'datetime'] = pd.to_datetime(data['datetime'])
data.loc[:, 'monthyear'] = pd.to_datetime(data['monthyear'])

# Проверка идентичности столбцов 'datetime' и 'monthyear'
(data['datetime'] - data['monthyear']).unique()

array([0], dtype='timedelta64[ns]')

Поскольку столбцы 'datetime' и 'monthyear' одинаковые, то один из них можно удалить.

In [16]:
# Удаление столбца 'monthyear'
data.drop(['monthyear'], axis=1, inplace=True)

Определим возраст животных на момент покидания приюта, используя столбцы 'date_of_birth' и 'datetime'.

In [17]:
# Возраст животных в годах на момент покидания приюта
data['age_outcome_years_2'] = ((data['datetime'] - data['date_of_birth']))/ np.timedelta64(1, 'Y')
data.head(5)

,animal_id,animal_type,breed,color,date_of_birth,datetime,name,outcome_subtype,outcome_type,sex_upon_outcome,age_outcome_years_1,age_outcome_years_2
0,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07,2014-07-22 16:04:00,NaN,Partner,Transfer,Intact Male,0.038462,0.042901
1,A666430,Dog,Beagle Mix,White/Brown,2012-11-06,2013-11-07 11:47:00,Lucy,Partner,Transfer,Spayed Female,1.000000,1.003418
2,A675708,Dog,Pit Bull,Blue/White,2013-03-31,2014-06-03 14:20:00,*Johnny,NaN,Adoption,Neutered Male,1.000000,1.176197
3,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02,2014-06-15 15:50:00,Monday,Partner,Transfer,Neutered Male,9.000000,9.036899
5,A664462,Dog,Leonberger Mix,Brown/White,2013-06-03,2013-10-07 13:06:00,*Edgar,Partner,Transfer,Intact Male,0.333333,0.346471


Проверим, чтобы возраст всех животных был больше 0.

In [19]:
# Количество и доля животных, возраст которых меньше 0
print('Количество животных с возрастом меньше нуля:', data.query('age_outcome_years_2 < 0')['age_outcome_years_2'].count())
print('Доля животных с возрастом меньше нуля:', data.query('age_outcome_years_2 < 0')['age_outcome_years_2'].count() / data['age_outcome_years_2'].count())


Количество животных с возрастом меньше нуля: 8
Доля животных с возрастом меньше нуля: 0.00010860121633362293


В результате получили 8 животных с возрастом меньше 0 по столбцу 'age_outcome_years_2'.

Столбцы 'age_outcome_years_1' и 'age_outcome_years_2' содержат в себе аналогичную информацию - возраст животного на момент покидания приюта. Поэтому для дальнейшего анализа нужно оставить только один из этих двух признаков. 
Сравним значения двух полученных столбцов: 'age_outcome_years_1' и 'age_outcome_years_2'.

In [20]:
# Сравнение значений столбцов 'age_outcome_years_1' и 'age_outcome_years_2'
df = pd.DataFrame({'type':['age_outcome_years_1', 'age_outcome_years_2'], 
      'nan_count':[data.isna().sum().age_outcome_years_1, data.isna().sum().age_outcome_years_2], 
      'negative_age_count':[data.query('age_outcome_years_1 < 0')['age_outcome_years_1'].count(), data.query('age_outcome_years_2 < 0')['age_outcome_years_2'].count()]})

df

,type,nan_count,negative_age_count
0,age_outcome_years_1,6,0
1,age_outcome_years_2,0,8


Как видно, в каждом столбце по 6 и 8 некорректных значений. При этом столбец 'age_outcome_years_2' содержит в себе более точные данные, поскольку рассчитывался с точностью до дня. 

Принимаем решение оставить столбец 'age_outcome_years_2', предварительно очистив его от отрицательного возраста (количество данных значений незначительно - менее < 1%)  . Столбец 'age_outcome_years_1' удаляем.

In [21]:
# Удаление отрицательного возраста из 'age_outcome_years_2'
data = data.query('age_outcome_years_2 >= 0')

# Удаление столбца 'age_outcome_years_1'
data.drop(['age_outcome_years_1'], axis=1, inplace=True)

data.head(5)

,animal_id,animal_type,breed,color,date_of_birth,datetime,name,outcome_subtype,outcome_type,sex_upon_outcome,age_outcome_years_2
0,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07,2014-07-22 16:04:00,NaN,Partner,Transfer,Intact Male,0.042901
1,A666430,Dog,Beagle Mix,White/Brown,2012-11-06,2013-11-07 11:47:00,Lucy,Partner,Transfer,Spayed Female,1.003418
2,A675708,Dog,Pit Bull,Blue/White,2013-03-31,2014-06-03 14:20:00,*Johnny,NaN,Adoption,Neutered Male,1.176197
3,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02,2014-06-15 15:50:00,Monday,Partner,Transfer,Neutered Male,9.036899
5,A664462,Dog,Leonberger Mix,Brown/White,2013-06-03,2013-10-07 13:06:00,*Edgar,Partner,Transfer,Intact Male,0.346471


#### 4. Столбец 'animal_id'.

#### 5. Столбцы 'color' и 'name'.

Исходя из допущения, что окрас животного и его кличка не влияют на значения столбца 'outcome_type', удалим данные признаки из датасета -  'color' и 'name'.

In [22]:
# Удаление столбцов 'color' и 'name'
data.drop(['color', 'name'], axis=1, inplace=True)
data.head(5)

,animal_id,animal_type,breed,date_of_birth,datetime,outcome_subtype,outcome_type,sex_upon_outcome,age_outcome_years_2
0,A684346,Cat,Domestic Shorthair Mix,2014-07-07,2014-07-22 16:04:00,Partner,Transfer,Intact Male,0.042901
1,A666430,Dog,Beagle Mix,2012-11-06,2013-11-07 11:47:00,Partner,Transfer,Spayed Female,1.003418
2,A675708,Dog,Pit Bull,2013-03-31,2014-06-03 14:20:00,NaN,Adoption,Neutered Male,1.176197
3,A680386,Dog,Miniature Schnauzer Mix,2005-06-02,2014-06-15 15:50:00,Partner,Transfer,Neutered Male,9.036899
5,A664462,Dog,Leonberger Mix,2013-06-03,2013-10-07 13:06:00,Partner,Transfer,Intact Male,0.346471


#### 6. Столбец 'sex_upon_outcome'.

In [23]:
data['sex_upon_outcome'].value_counts()

Neutered Male    27690
Spayed Female    25117
Intact Male       9182
Intact Female     8902
Unknown           2763
Name: sex_upon_outcome, dtype: int64

Столбец 'sex_upon_outcome' обозначает пол животного, а также, было ли оно стерилизовано на момент покидания приюта. Этот параметр можно разделить на два новых: пол и Spay / Neuter.

In [24]:
# Выделение пола животного
data.loc[ data['sex_upon_outcome'].isin(['Intact Male', 'Neutered Male']), 'sex_type' ] = 'male'
data.loc[ data['sex_upon_outcome'].isin(['Spayed Female', 'Intact Female']), 'sex_type' ] = 'female'
data.head(5)

,animal_id,animal_type,breed,date_of_birth,datetime,outcome_subtype,outcome_type,sex_upon_outcome,age_outcome_years_2,sex_type
0,A684346,Cat,Domestic Shorthair Mix,2014-07-07,2014-07-22 16:04:00,Partner,Transfer,Intact Male,0.042901,male
1,A666430,Dog,Beagle Mix,2012-11-06,2013-11-07 11:47:00,Partner,Transfer,Spayed Female,1.003418,female
2,A675708,Dog,Pit Bull,2013-03-31,2014-06-03 14:20:00,NaN,Adoption,Neutered Male,1.176197,male
3,A680386,Dog,Miniature Schnauzer Mix,2005-06-02,2014-06-15 15:50:00,Partner,Transfer,Neutered Male,9.036899,male
5,A664462,Dog,Leonberger Mix,2013-06-03,2013-10-07 13:06:00,Partner,Transfer,Intact Male,0.346471,male


In [25]:
print(f'Пол неопределен: {data.sex_type.isna().sum()} животных или {round(100 * data.sex_type.isna().sum() / data.sex_type.count(), 1)}%')

Пол неопределен: 2765 животных или 3.9%


In [26]:
# Определение Spay / Neuter животного
data.loc[ data['sex_upon_outcome'].isin(['Intact Female', 'Intact Male']), 'spay/neuter' ] = 'no'
data.loc[ data['sex_upon_outcome'].isin(['Spayed Female', 'Neutered Male']), 'spay/neuter' ] = 'yes'

# Удаление столбца 'sex_upon_outcome'
data.drop(['sex_upon_outcome'], axis=1, inplace=True)
data.head()


,animal_id,animal_type,breed,date_of_birth,datetime,outcome_subtype,outcome_type,age_outcome_years_2,sex_type,spay/neuter
0,A684346,Cat,Domestic Shorthair Mix,2014-07-07,2014-07-22 16:04:00,Partner,Transfer,0.042901,male,no
1,A666430,Dog,Beagle Mix,2012-11-06,2013-11-07 11:47:00,Partner,Transfer,1.003418,female,yes
2,A675708,Dog,Pit Bull,2013-03-31,2014-06-03 14:20:00,NaN,Adoption,1.176197,male,yes
3,A680386,Dog,Miniature Schnauzer Mix,2005-06-02,2014-06-15 15:50:00,Partner,Transfer,9.036899,male,yes
5,A664462,Dog,Leonberger Mix,2013-06-03,2013-10-07 13:06:00,Partner,Transfer,0.346471,male,no


## Кошки

In [89]:
cat = data.loc[ data['animal_type'].isin(['Cat'])]
cat.breed.value_counts()

Domestic Shorthair Mix                     23330
Domestic Medium Hair Mix                    2323
Domestic Longhair Mix                       1228
Siamese Mix                                  998
Domestic Shorthair                           386
                                           ...  
Domestic Shorthair/Domestic Shorthair          1
Domestic Longhair/Domestic Longhair            1
Domestic Shorthair/Domestic Medium Hair        1
Domestic Shorthair/Maine Coon                  1
Domestic Shorthair/Manx                        1
Name: breed, Length: 79, dtype: int64

Анализ показал, что домашняя порода является наиболее распространенной. 

Поскольку слово 'Mix' встречается в большенстве названий пород, то предположим, что этот термин не относится к типу пород и, удалим его из названия пород.

In [90]:
cat['breed'] = cat['breed'].str.lower().str.replace('mix', '').str.replace('medium hair', 'mediumhair').str.strip()
cat.tail(50)
cat.breed.unique()


C:\Users1\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


array(['domestic shorthair', 'domestic mediumhair', 'siamese',
       'russian blue', 'domestic longhair', 'manx', 'ragdoll',
       'snowshoe/domestic shorthair', 'snowshoe', 'angora', 'himalayan',
       'domestic longhair/persian', 'japanese bobtail',
       'domestic longhair/rex', 'siamese/domestic shorthair',
       'domestic mediumhair/siamese', 'maine coon', 'devon rex',
       'balinese', 'american shorthair', 'british shorthair',
       'angora/persian', 'munchkin shorthair',
       'domestic shorthair/siamese', 'manx/domestic longhair', 'persian',
       'cymric', 'tonkinese', 'siamese/angora', 'burmese', 'sphynx',
       'domestic shorthair/domestic mediumhair', 'bengal',
       'domestic longhair/russian blue', 'bombay', 'exotic shorthair',
       'domestic shorthair/british shorthair', 'abyssinian',
       'manx/domestic shorthair', 'norwegian forest cat',
       'snowshoe/ragdoll', 'manx/siamese', 'turkish van', 'cornish rex',
       'birman', 'american curl shorthair', 

Ассоциация любителей кошек (CFA) в настоящее время признает 42 породы кошек. Поэтому создадим два новых признака, которые отличают кошку от породы, признанной CFA, или домашней породы. А далее проанализируем, влияет ли порода кошек на итоговый результат работы.

Создадим список всех пород CFA, а затем преобразуем этот список в нижний регистр и соединяем все элементы с панелью.

In [91]:
# Список всех пород кошек
cfa_breeds = ['Abyssinian', 'American Bobtail', 'American Curl', 'American Shorthair', 'American Wirehair', 
              'Balinese', 'Bengal', 'Birman', 'Bombay', 'British Shorthair', 'Burmese', 'Burmilla', 
              'Chartreux', 'Colorpoint Shorthair', 'Cornish Rex', 'Devon Rex', 'Egyptian Mau', 'European Burmese', 
              'Exotic', 'Havana Brown', 'Japanese Bobtail', 'Korat', 'LaPerm', 'Maine Coon', 'Manx', 
              'Norwegian Forest Cat', 'Ocicat', 'Oriental', 'Persian', 'Ragamuffin', 'Ragdoll', 'Russian Blue', 
              'Scottish Fold', 'Selkirk Rex', 'Siamese', 'Siberian', 'Singapura', 'Somali', 'Tonkinese', 
              'Turkish Angora', 'Turkish Van']


cfa_breeds = [i.lower() for i in cfa_breeds]
cfa_breeds = '|'.join(cfa_breeds)
cfa_breeds

'abyssinian|american bobtail|american curl|american shorthair|american wirehair|balinese|bengal|birman|bombay|british shorthair|burmese|burmilla|chartreux|colorpoint shorthair|cornish rex|devon rex|egyptian mau|european burmese|exotic|havana brown|japanese bobtail|korat|laperm|maine coon|manx|norwegian forest cat|ocicat|oriental|persian|ragamuffin|ragdoll|russian blue|scottish fold|selkirk rex|siamese|siberian|singapura|somali|tonkinese|turkish angora|turkish van'

In [92]:
cat['breed'].str.split('/', expand=True).rename(columns={0:'breed1', 1:'breed2'})

cat = pd.concat([cat, 
                     cat['breed'].str.split('/', expand=True).rename(columns={0:'breed1', 1:'breed2'})], 
                    axis=1)

cat['cfa_breed'] = np.where((cat['breed1'].str.contains(cfa_breeds)) | 
                                (cat['breed2'].str.contains(cfa_breeds)), 
                                True, False)

cat['breeds'] = np.where((cat['breed1'].str.contains(cfa_breeds) == False) | 
                                     (cat['breed2'].str.contains(cfa_breeds) == False), 
                                     'no', 'yes')


# Удаление столбов 'breed', 'breed1', 'breed2' и 'cfa_breed'
cat.drop(['breed', 'breed1', 'breed2', 'cfa_breed'], axis=1, inplace=True)
cat.head()

cat.head(5)

,animal_id,animal_type,date_of_birth,datetime,outcome_subtype,outcome_type,age_outcome_years_2,sex_type,spay/neuter,breeds
0,A684346,Cat,2014-07-07,2014-07-22 16:04:00,Partner,Transfer,0.042901,male,no,no
8,A685067,Cat,2014-06-16,2014-08-14 18:45:00,NaN,Adoption,0.163676,female,no,no
9,A678580,Cat,2014-03-26,2014-06-29 17:45:00,Offsite,Adoption,0.262126,female,yes,no
10,A675405,Cat,2013-03-27,2014-03-28 14:55:00,NaN,Return to Owner,1.003776,female,yes,no
17,A670420,Cat,2013-12-16,2014-01-09 19:29:00,Partner,Transfer,0.067932,male,no,no
19,A684460,Cat,2014-06-02,2014-08-13 15:04:00,NaN,Adoption,0.198848,female,yes,no
20,A673952,Cat,2013-07-05,2014-03-06 14:29:00,SCRP,Transfer,0.669702,NaN,NaN,no
21,A686497,Cat,2014-03-22,2014-08-31 15:45:00,NaN,Adoption,0.445338,male,yes,no
23,A687965,Cat,2014-08-03,2014-10-31 18:29:00,Foster,Adoption,0.245782,male,yes,no
24,A668547,Cat,2012-12-04,2013-12-16 13:59:00,Partner,Transfer,1.033786,male,yes,no


In [126]:
cat.breeds.count()

29417

## СОБАКИ

Проанализируем породы собак в датасете. 

In [121]:
# Список пород собак
dog = data.loc[ data['animal_type'].isin(['Dog'])]
list(dog.breed.unique())

['Beagle Mix',
 'Pit Bull',
 'Miniature Schnauzer Mix',
 'Leonberger Mix',
 'Chihuahua Shorthair Mix',
 'Papillon/Border Collie',
 'Chihuahua Shorthair/Pomeranian',
 'Miniature Schnauzer/Miniature Poodle',
 'Labrador Retriever Mix',
 'Rat Terrier Mix',
 'Pit Bull Mix',
 'German Shepherd/Labrador Retriever',
 'Mastiff Mix',
 'Plott Hound Mix',
 'Miniature Pinscher Mix',
 'Cardigan Welsh Corgi/English Setter',
 'Border Collie Mix',
 'Boxer Mix',
 'Parson Russell Terrier Mix',
 'Miniature Poodle Mix',
 'Australian Shepherd/Australian Cattle Dog',
 'Basenji/Chihuahua Longhair',
 'Labrador Retriever',
 'Siberian Husky/German Shepherd',
 'German Shepherd/Boxer',
 'Labrador Retriever/German Shepherd',
 'German Shepherd/Chow Chow',
 'Pit Bull/Labrador Retriever',
 'Yorkshire Terrier Mix',
 'Beagle/Chihuahua Longhair',
 'Parson Russell Terrier/Chihuahua Shorthair',
 'German Shepherd Mix',
 'Staffordshire',
 'Catahoula/Labrador Retriever',
 'Shih Tzu Mix',
 'Dachshund Mix',
 'American Bulldog Mi

Поскольку из предоставленного датасета точно не известно, какие собаки породистые, а какие помесь, то будем исходить из допущения, что значение столбца 'beed', в котором встречаются две породы, разделенные '/', относится к непородистым собакам.  

Создадим новый признак 'dog_breed', который отличает собаку как породистую и как помесь разных пород.

In [122]:
# Новый признак 'dog_breed': значение 'no', если собака не породистая
dog.loc[ dog['breed'].str.contains('/'), 'dog_breed' ] = 'no'

Поскольку изначально в столбце 'breed' не было пустых значений, то ячейки со значением Nan будут соответствовать породистым собакам.

In [123]:
# Заполнение ячеек Nan столбца 'dog_breed'
dog['dog_breed'].fillna('yes', inplace=True)
dog.head(5)

# dog['dog_breed'] = np.where((dog['breed'].str.contains('/')), 'yes', 'no') - альтернатива предыдущих двух команд


# Удаление столбца 'breed'
dog.drop(['breed'], axis=1, inplace=True)
dog.head()

# Переименование столбца 'dog_breed' в 'breed' 
dog.rename(columns={'dog_breed': 'breeds'}, inplace=True)
dog.head()

,animal_id,animal_type,date_of_birth,datetime,outcome_subtype,outcome_type,age_outcome_years_2,sex_type,spay/neuter,breeds
1,A666430,Dog,2012-11-06,2013-11-07 11:47:00,Partner,Transfer,1.003418,female,yes,yes
2,A675708,Dog,2013-03-31,2014-06-03 14:20:00,NaN,Adoption,1.176197,male,yes,yes
3,A680386,Dog,2005-06-02,2014-06-15 15:50:00,Partner,Transfer,9.036899,male,yes,yes
5,A664462,Dog,2013-06-03,2013-10-07 13:06:00,Partner,Transfer,0.346471,male,no,yes
7,A692618,Dog,2011-11-23,2014-12-08 15:55:00,Partner,Transfer,3.043630,female,yes,yes


In [125]:
# Объединение датафреймов cat и dog
data_dc = cat.append(dog, ignore_index=True)
data_dc

,animal_id,animal_type,date_of_birth,datetime,outcome_subtype,outcome_type,age_outcome_years_2,sex_type,spay/neuter,breeds
0,A684346,Cat,2014-07-07,2014-07-22 16:04:00,Partner,Transfer,0.042901,male,no,no
1,A685067,Cat,2014-06-16,2014-08-14 18:45:00,NaN,Adoption,0.163676,female,no,no
2,A678580,Cat,2014-03-26,2014-06-29 17:45:00,Offsite,Adoption,0.262126,female,yes,no
3,A675405,Cat,2013-03-27,2014-03-28 14:55:00,NaN,Return to Owner,1.003776,female,yes,no
4,A670420,Cat,2013-12-16,2014-01-09 19:29:00,Partner,Transfer,0.067932,male,no,no
...,...,...,...,...,...,...,...,...,...,...
73651,A764895,Dog,2017-12-04,2018-02-01 18:40:00,Foster,Adoption,0.163666,male,yes,no
73652,A764894,Dog,2017-12-04,2018-02-01 18:26:00,Foster,Adoption,0.163639,female,yes,no
73653,A764468,Dog,2014-12-30,2018-02-01 18:06:00,NaN,Adoption,3.093162,male,yes,yes
73654,A765858,Dog,2017-11-13,2018-02-01 18:32:00,NaN,Adoption,0.221147,female,yes,yes


In [133]:
# Удаление столбца 'animal_id'
data_dc.drop(['animal_id'], axis=1, inplace=True)
data_dc.head()

# data_dc.animal_id.value_counts() М.б. сделать 2 группы: 1)один раз попадали в приют; 2) более одного раза попадали в приют

,animal_type,date_of_birth,datetime,outcome_subtype,outcome_type,age_outcome_years_2,sex_type,spay/neuter,breeds
0,Cat,2014-07-07,2014-07-22 16:04:00,Partner,Transfer,0.042901,male,no,no
1,Cat,2014-06-16,2014-08-14 18:45:00,NaN,Adoption,0.163676,female,no,no
2,Cat,2014-03-26,2014-06-29 17:45:00,Offsite,Adoption,0.262126,female,yes,no
3,Cat,2013-03-27,2014-03-28 14:55:00,NaN,Return to Owner,1.003776,female,yes,no
4,Cat,2013-12-16,2014-01-09 19:29:00,Partner,Transfer,0.067932,male,no,no


In [ ]:
1. age_upon_outcome - возрст животного на момент попадания в приют
2. animal_id - не все уникальные, количество повторов достигает 11
3. breed - порода
4. datetime и monthyear - дата и время поступления
5. sex_upon_outcome - пол животного и, если оно было стерилизовано или стерилизовано.

6. sex_upon_outcome - разделить на два признака: пол и стерил/нестерил
7. разделить на две категории: порода - непорода
    8. возраст - в неудобном для анализа формате. нужно преобразовать

In [ ]:
data.outcome_type.value_counts()
# len(data.animal_id.unique())